In [7]:
import xarray as xr

In [62]:
with xr.open_dataset("../../test_data/CPEXAW-DROPSONDE_D20210806_193025_PQC.nc", decode_cf=False) as ds:
        print(ds.data_vars)
        print("----------")
        for key in ds.attrs:
                print(key)

Data variables:
    trajectory      |S1 ...
    launch_time     int32 ...
    pres            (time) float32 ...
    tdry            (time) float32 ...
    dp              (time) float32 ...
    rh              (time) float32 ...
    u_wind          (time) float32 ...
    v_wind          (time) float32 ...
    w_wind          (time) float32 ...
    wspd            (time) float32 ...
    wdir            (time) float32 ...
    dz              (time) float32 ...
    mr              (time) float32 ...
    vt              (time) float32 ...
    theta           (time) float32 ...
    theta_e         (time) float32 ...
    theta_v         (time) float32 ...
    lat             (time) float32 ...
    lon             (time) float32 ...
    alt             (time) float32 ...
    gpsalt          (time) float32 ...
    reference_time  (obs) int32 ...
    reference_pres  (obs) float32 ...
    reference_tdry  (obs) float32 ...
    reference_rh    (obs) float32 ...
    reference_wspd  (obs) float32 .

In [101]:
with xr.open_dataset("../../test_data/CPEXAW-DROPSONDE_D20210820_203208_PQC.nc", decode_cf=False) as ds:
        rh = ds['rh'].values # relative humidity
        dp = ds['dp'].values # dew point
        tdry = ds['tdry'].values # temp dry???
        lat = ds['lat'].values
        lon = ds['lon'].values
        alt = ds['alt'].values
        time = ds['time'].values

        print(len(rh), "-", (rh.shape)) #np ndarray
        print(len(dp), "-", (dp.shape)) 
        print(len(tdry), "-", (tdry.shape)) 
        print(len(lat), "-", (lat.shape))
        print(len(lon), "-", (lon.shape))
        print(len(alt), "-", (alt.shape))   
        print(len(time), "-", (time.shape)) 

        # print(time, len(time)) 
        print(lat[123])
        print(lon[123])
        print(alt[123])   


2979 - (2979,)
2979 - (2979,)
2979 - (2979,)
2979 - (2979,)
2979 - (2979,)
2979 - (2979,)
2979 - (2979,)
-999.0
-999.0
-999.0


netcdf to zarr

In [127]:
import zarr
import numpy as np
import xarray as xr

# META needed for ingest
campaign = 'Olympex'
collection = "AirborneRadar"
dataset = "gpmValidationOlympexcrs"
variables = ["zku"]
renderers = ["point_cloud"]
chunk = 262144
to_rad = np.pi / 180
to_deg = 180 / np.pi

def ingest(folder, file, s3bucket):
    """
    Converts Level 1B crs data from s3 to zarr file and then stores it in the provided folder
    Args:
        folder (string): name to hold the raw files.
        file (string): the s3 url to the raw file.
    """
    store = zarr.DirectoryStore(folder)
    root = zarr.group(store=store)
    
    # Create empty rows for modified data    
    z_chunk_id = root.create_dataset('chunk_id', shape=(0, 2), chunks=None, dtype=np.int64)
    z_location = root.create_dataset('location', shape=(0, 3), chunks=(chunk, None), dtype=np.float32)
    z_time = root.create_dataset('time', shape=(0), chunks=(chunk), dtype=np.int32)
    z_vars = root.create_group('value')
    z_ref = z_vars.create_dataset('ref', shape=(0), chunks=(chunk), dtype=np.float32)
    n_time = np.array([], dtype=np.int64)

    # date = file.split("_")[2]
    date = ""
    base_time = ""
    # base_time = np.datetime64('{}-{}-{}'.format(date[:4], date[4:6], date[6:]))

    # open dataset.
    with xr.open_dataset("../../test_data/CPEXAW-DROPSONDE_D20210820_203208_PQC.nc", decode_cf=False) as ds:
        rh = ds['rh'].values # relative humidity
        dp = ds['dp'].values # dew point
        tdry = ds['tdry'].values # temp dry???
        lat = ds['lat'].values
        lon = ds['lon'].values
        alt = ds['alt'].values
        time = ds['time'].values

    # data formation
    
    ref = np.column_stack((rh, dp, tdry)).reshape(-1)    
    # as 3 kind of data at a single point in 3d space(lon lat alt) in a given time
    lon = np.repeat(lon, 3)
    lat = np.repeat(lat, 3)
    alt = np.repeat(alt, 3)
    time = np.repeat(time, 3)
    
    print(ref.shape)
    print(lon.shape)
    print(lat.shape)
    print(alt.shape)
    print(time.shape)
    
    # sort data by time
    sort_idx = np.argsort(time)

    lon = lon[sort_idx]
    lat = lat[sort_idx]
    alt = alt[sort_idx]
    ref = ref[sort_idx]
    time = time[sort_idx]

    # remove nan and infinite using mask ???
    mask = np.logical_and(alt != -999.0, lon != -999.0, lat != -999.0)
    # mask = np.logical_and(np.isfinite(ref), alt > 0, alt != -999.0, lon != -999.0, lat != -999.0)
    lon = lon[mask]
    lat = lat[mask]
    alt = alt[mask]
    ref = ref[mask]
    time = time[mask]

    # Now populate (append) the empty rows with modified data.
    z_location.append(np.stack([lon, lat, alt], axis=-1))
    z_ref.append(ref)
    n_time = np.append(n_time, time)

    idx = np.arange(0, n_time.size, chunk)
    chunks = np.zeros(shape=(idx.size, 2), dtype=np.int64)
    chunks[:, 0] = idx
    chunks[:, 1] = n_time[idx]
    z_chunk_id.append(chunks)

    epoch = np.min(n_time)
    n_time = (n_time - epoch).astype(np.int32)
    z_time.append(n_time)

    # save it.
    root.attrs.put({
        "campaign": campaign,
        "collection": collection,
        "dataset": dataset,
        "variables": variables,
        "renderers": renderers,
        "epoch": int(epoch)

    })

In [128]:
zarr_folder = "./tmp/crs_olympex/zarr/dropsonde"
ingest(zarr_folder, "", "")

(8937,)
(8937,)
(8937,)
(8937,)
(8937,)


Generate point cloud"

In [129]:
import os
import json
import numpy as np
from datetime import datetime
from threading import Thread, Lock

to_rad = np.pi / 180.0
to_deg = 180.0 / np.pi

steps = [32, 16, 8, 4, 2, 1]

class PointCloud:
    def __init__(self, key, lon, lat, alt, value, time, epoch):
        self.key = key
        self.lon = lon
        self.lat = lat
        self.alt = alt
        self.time = time
        self.value = value
        self.epoch = epoch
        self.tasks = []
        self.threads = []
        for i in range(10):
            self.threads.append(Thread(target=self.worker_function))
        self.tileset_lock = Lock()
        self.tileset_json = {
        	"asset": {
        		"version": "1.0",
        		"type": "Airborne Radar"
        	},
        	"root": {
        		"geometricError": 1000000,
        		"refine" : "REPLACE",
        		"boundingVolume": {
                    "region": [
                        float(np.min(lon)) * to_rad,
                        float(np.min(lat)) * to_rad,
                        float(np.max(lon)) * to_rad,
                        float(np.max(lat)) * to_rad,
                        float(np.min(alt)) * to_rad,
                        float(np.max(alt)) * to_rad
                    ]
                },
                "children": []
        	},
            "properties": {
                "epoch": "{}Z".format(datetime.utcfromtimestamp(epoch).isoformat()),
                "refined": []
            }
        }


    def worker_function(self):
        while len(self.tasks) > 0:
                tile, start, end = self.tasks.pop()
                print(tile, start, end)
                self.generate(tile, start, end)


    def start(self):
        for t in self.threads:
            t.start()


    def join(self):
        for t in self.threads:
            t.join()
        with open('{}/tileset.json'.format(self.key), mode='w+') as outfile:
            json.dump(self.tileset_json, outfile)


    def schedule_task(self, tile, start, end):
        self.tasks.append((tile, start, end))


    def generate(self, tile, start, end):
        print(tile, start, end)
        parent_tile = self.tileset_json["root"]
        cartesian, offset, scale, cartographic, region = self.cartographic_to_cartesian(start, end)

        value = self.value[start:end]
        time = self.time[start:end]

        epoch = int(np.min(time) + self.epoch - 300)
        epoch = "{}Z".format(datetime.utcfromtimestamp(epoch).isoformat())
        end = int(np.max(time) + self.epoch + 300)
        end = "{}Z".format(datetime.utcfromtimestamp(end).isoformat())

        header_length = 28
        magic = np.string_("pnts")
        version = 1

        for step in steps:
            self.tileset_lock.acquire()
            try:
                filename = "{}_{}.pnts".format(tile, step)
                child_tile = {
                    "availability": "{}/{}".format(epoch, end),
                    "geometricError": step * 500,
                    "boundingVolume": {
                        "region": region
                    },
                    "content": {
                        "uri": filename
                    },
                    "refine": "REPLACE"
                }
                if step == 1:
                    self.tileset_json["properties"]["refined"].append(filename)
                else:
                    child_tile["children"] = []
                parent_tile["children"].append(child_tile)
                parent_tile = child_tile
            finally:
                self.tileset_lock.release()

            tile_length = 0
            feature_table_binary_byte_length = 0
            batch_table_binary_byte_length = 0
            length = value[::step].size

            feature_table_json = {
                "POINTS_LENGTH": length,
                "BATCH_LENGTH": length,
                "BATCH_ID": {
                    "byteOffset": 0,
                    "componentType": "UNSIGNED_INT"
                },
                "POSITION_QUANTIZED": {
                    "byteOffset": length * 4
                },
                "QUANTIZED_VOLUME_OFFSET": offset,
                "QUANTIZED_VOLUME_SCALE": scale
            }

            batch_table_json = {
                "value": {
                    "byteOffset": 0,
                    "componentType": "FLOAT",
                    "type": "SCALAR"
                },
                "time": {
                    "byteOffset": length * 4,
                    "componentType": "FLOAT",
                    "type": "SCALAR"
                },
                "location": {
                    "byteOffset": length * 8,
                    "componentType": "SHORT",
                    "type": "VEC3"
                }
            }

            tile_length += header_length

            feature_table_json_min = json.dumps(feature_table_json, separators=(",", ":")) + "       "
            feature_table_trim = (tile_length + len(feature_table_json_min)) % 8
            if feature_table_trim != 0:
                feature_table_json_min = feature_table_json_min[:-feature_table_trim]

            tile_length += len(feature_table_json_min)

            feature_table_binary_byte_length = length * 4 + length * 3 * 2
            tile_length += feature_table_binary_byte_length
            feature_table_padding = tile_length % 8
            if feature_table_padding != 0:
                feature_table_padding = 8 - feature_table_padding
            tile_length += feature_table_padding

            batch_table_json_min = json.dumps(batch_table_json, separators=(",", ":")) + "       "
            batch_table_trim = (tile_length + len(batch_table_json_min)) % 8
            if batch_table_trim != 0:
                batch_table_json_min = batch_table_json_min[:-batch_table_trim]

            tile_length += len(batch_table_json_min)

            batch_table_binary_byte_length = length * 4 * 2 + length * 2 * 3
            tile_length += batch_table_binary_byte_length
            batch_table_padding = tile_length % 8
            if batch_table_padding != 0:
                batch_table_padding = 8 - batch_table_padding
            tile_length += batch_table_padding

            with open('{}/{}'.format(self.key, filename), mode='wb+') as outfile:
                outfile.write(np.string_(magic).tobytes())
                outfile.write(np.uint32(version).tobytes())
                outfile.write(np.uint32(tile_length).tobytes())
                outfile.write(np.uint32(len(feature_table_json_min)).tobytes())
                outfile.write(np.uint32(feature_table_binary_byte_length + feature_table_padding).tobytes())
                outfile.write(np.uint32(len(batch_table_json_min)).tobytes())
                outfile.write(np.uint32(batch_table_binary_byte_length + batch_table_padding).tobytes())
                outfile.write(np.string_(feature_table_json_min).tobytes())
                outfile.write(np.arange(length, dtype=np.uint32).tobytes())
                outfile.write(cartesian[::step, :].tobytes())
                for _ in range(feature_table_padding):
                    outfile.write(np.string_(" ").tobytes())
                outfile.write(np.string_(batch_table_json_min).tobytes())
                outfile.write(value[::step].astype(np.float32).tobytes())
                outfile.write(time[::step].astype(np.float32).tobytes())
                outfile.write(cartographic[::step, :].tobytes())
                for _ in range(batch_table_padding):
                    outfile.write(np.string_(" ").tobytes())
                outfile.seek(0)


    def cartographic_to_cartesian(self, start, end):
        lon = self.lon[start:end]
        lat = self.lat[start:end]
        alt = self.alt[start:end]
        size = lon.size

        cartographic = np.zeros(shape=(size, 3), dtype=np.int16)
        cartographic[:, 0] = (lon * 32767 / 180).astype(np.int16)
        cartographic[:, 1] = (lat * 32767 / 180).astype(np.int16)
        cartographic[:, 2] = (alt / 10).astype(np.int16)

        lon = lon * to_rad
        lat = lat * to_rad

        radiiSquared = np.array([40680631590769, 40680631590769, 40408299984661.445], dtype=np.float64)

        N1 = np.multiply(np.cos(lat), np.cos(lon))
        N2 = np.multiply(np.cos(lat), np.sin(lon))
        N3 = np.sin(lat)

        magnitude = np.sqrt(np.square(N1) + np.square(N2) + np.square(N3))

        N1 = N1 / magnitude
        N2 = N2 / magnitude
        N3 = N3 / magnitude

        K1 = radiiSquared[0] * N1
        K2 = radiiSquared[1] * N2
        K3 = radiiSquared[2] * N3

        gamma = np.sqrt(np.multiply(N1, K1) + np.multiply(N2, K2) + np.multiply(N3, K3))

        K1 = K1 / gamma
        K2 = K2 / gamma
        K3 = K3 / gamma

        N1 = np.multiply(N1, alt)
        N2 = np.multiply(N2, alt)
        N3 = np.multiply(N3, alt)

        # x = np.multiply((N1 + K1), np.random.normal(1, .00005, N1.size))
        # y = np.multiply((N2 + K2), np.random.normal(1, .00005, N1.size))
        # z = np.multiply((N3 + K3), np.random.normal(1, .00005, N1.size))

        x = N1 + K1
        y = N2 + K2
        z = N3 + K3

        offset = [float(np.min(x)), float(np.min(y)), float(np.min(z))]

        x = x - offset[0]
        y = y - offset[1]
        z = z - offset[2]

        scale = [float(abs(np.max(x))), float(abs(np.max(y))), float(abs(np.max(z)))]

        cartesian = np.zeros(shape=(size, 3), dtype=np.uint16)
        cartesian[:, 0] = (x / scale[0] * 65535.0).astype(np.uint16)
        cartesian[:, 1] = (y / scale[1] * 65535.0).astype(np.uint16)
        cartesian[:, 2] = (z / scale[2] * 65535.0).astype(np.uint16)

        region = [
            float(np.min(lon)),
            float(np.min(lat)),
            float(np.max(lon)),
            float(np.max(lat)),
            float(np.min(alt)),
            float(np.max(alt))
        ]

        return cartesian, offset, scale, cartographic, region


In [130]:
import sys
import os
import zarr
import numpy as np
import json
import datetime as dt

to_rad = np.pi / 180.0
to_deg = 180.0 / np.pi

def generate_point_cloud(variable, epoch, end, zarr_location, point_cloud_folder):
    """Generates json pointcloud from a given zarr file input

    Args:
        variable (_type_): _description_
        epoch (_type_): _description_
        end (_type_): _description_
        zarr_location (string): source zarr file.
        point_cloud_folder (string): destination folder for 3d tile json file.
    """

    #out_key = f"{os.getenv('CRS_OUTPUT_FLIGHT_PATH')}/{shortname}"
    #pc_out_key = f"{output_path}/point_cloud"

    '''
    try:
        os.mkdir(out_key)
    except:
        pass
    '''

    try:
        os.mkdir(point_cloud_folder)
    except:
        pass

    # LOAD THE DATA.
    store = zarr.DirectoryStore(zarr_location)
    root = zarr.group(store=store)

    chunk_id = root["chunk_id"][:]
    num_chunks = chunk_id.shape[0]
    id = np.argmax(chunk_id[:, 1] > epoch) - 1
    start_id = chunk_id[0 if id < 0 else id, 0]
    id = num_chunks - np.argmax(chunk_id[::-1, 1] < end)
    end_id = chunk_id[id, 0] if id < num_chunks else root["time"].size - 1

    root_epoch = root.attrs["epoch"]
    location = root["location"][start_id:end_id]
    lon = location[:, 0]
    lat = location[:, 1]
    alt = location[:, 2]
    value = root["value"][variable][start_id:end_id]
    time = root["time"][start_id:end_id]

    # filter data using mask
    epoch = epoch - root_epoch # date-time
    end = end - root_epoch
    mask = np.logical_and(time >= epoch, time <= end)
    lon = lon[mask]
    lat = lat[mask]
    alt = alt[mask]
    value = value[mask]
    time = time[mask]

    # Generate Pointcloud Tileset
    point_cloud = PointCloud(point_cloud_folder, lon, lat, alt, value, time, root_epoch)

    for tile in range(int(np.ceil(time.size / 530000))):
        start_id = tile * 530000
        end_id = np.min([start_id + 530000, time.size])
        point_cloud.schedule_task(tile, start_id, end_id)

    point_cloud.start()
    point_cloud.join()

tileset_json = {
	"asset": {
		"version": "1.0",
		"type": "Airborne Radar"
	},
	"root": {
		"geometricError": 1000000,
		"refine" : "REPLACE",
		"boundingVolume": {
            "region": []
        },
        "children": []
	},
    "properties": {
        "epoch": "",
        "refined": []
    }
}

#if __name__ == '__main__':
#    main(sys.argv[1], sys.argv[2], int(sys.argv[3]), int(sys.argv[4]))


In [131]:
point_cloud_folder = f"{zarr_folder}/point_cloud"
generate_point_cloud("ref",  0,  1000000000000, zarr_folder, point_cloud_folder)


0 0 2474
0 0 2474


TEST

In [111]:
import numpy as np
import pandas as pd
np.random.seed(0)
temperature = 15 + 8 * np.random.randn(2, 2, 3)
precipitation = 10 * np.random.rand(2, 2, 3)
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]
time = pd.date_range("2014-09-06", periods=3)
reference_time = pd.Timestamp("2014-09-05")
print(temperature.shape)
print(precipitation)
print(lon)
print(lat)
print(time)
print(reference_time)

(2, 2, 3)
[[[5.68044561 9.25596638 0.71036058]
  [0.871293   0.20218397 8.32619846]]

 [[7.78156751 8.70012148 9.78618342]
  [7.99158564 4.61479362 7.80529176]]]
[[-99.83, -99.32], [-99.79, -99.23]]
[[42.25, 42.21], [42.63, 42.59]]
DatetimeIndex(['2014-09-06', '2014-09-07', '2014-09-08'], dtype='datetime64[ns]', freq='D')
2014-09-05 00:00:00


In [114]:
x = np.array([1,2,3])
print(x)
xx = np.repeat(x, 3)
print(xx)

[1 2 3]
[1 1 1 2 2 2 3 3 3]


In [133]:
a = np.array([1,2,3])
b = np.array([4,5,6])
c = np.array([7,8,9])
res = np.column_stack((a,b,c)).reshape(-1)
res

array([1, 4, 7, 2, 5, 8, 3, 6, 9])